# Agents
agent的核心思想是使用LLM来选择要采取的一系列动作。 在链式结构中，一系列动作是硬编码的（在代码中）。 在agent中，使用语言模型作为推理引擎来确定要采取的动作及其顺序。

这里有几个关键组件：
## 代理
这是负责决定下一步采取什么动作的类。 这是由语言模型和提示驱动的。 该提示可以包括以下内容：

代理的个性（对于以某种方式响应很有用）
代理的背景上下文（对于给予其更多关于所要求完成的任务类型的上下文很有用）
调用更好推理的提示策略（最著名/广泛使用的是ReAct）

## 工具 
工具是代理调用的函数。 这里有两个重要的考虑因素：

给代理访问正确工具的权限
以对代理最有帮助的方式描述工具
如果没有这两者，您想要构建的代理将无法工作。 如果您不给代理访问正确工具的权限，它将永远无法完成目标。 如果您不正确描述工具，代理将不知道如何正确使用它们。

## 工具包
代理可以访问的工具集合通常比单个工具更重要。 为此，LangChain提供了工具包的概念-用于实现特定目标所需的一组工具。 通常一个工具包中有3-5个工具。

In [2]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchResults
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()


True

In [39]:
# Create the agent
memory = MemorySaver()
model = ChatOllama(
    model="Solar",
    temperature=0.7,
    base_url="http://localhost:11434",
    stream_usage=True  # 流模式需开启统计
)

# search = DuckDuckGoSearchResults()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)


In [40]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()



================================ Human Message =================================

hi im bob! and i live in sf


ResponseError: registry.ollama.ai/library/Solar:latest does not support tools (status code: 400)

In [3]:
from langchain_ollama import ChatOllama

def simple_calculator(opertion: str, x:float, y:float)-> float:
    if opertion == "add":
        return x + y
    elif opertion == "subtract":
        return x - y
    elif opertion == "multiply":
        return x * y
    elif opertion == "divide":
        return x / y
    else:
        raise ValueError("Invalid operation")


In [4]:
llm_with_tools = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0.3,
    base_url = "http://10.1.4.136:11434/"
).bind_tools([simple_calculator])

result = llm_with_tools.invoke(
    "你知道一万乘153是多少吗"
)

print("tool resul",result.toolcalls)

ResponseError: registry.ollama.ai/library/deepseek-r1:32b does not support tools (status code: 400)

In [41]:
from langchain.agents import initialize_agent
# Create the agentfrom langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "Solar"
model  = ChatOllama(model=model_name)

# 定义工具
#export TAVILY_API_KEY="..."
search = TavilySearchResults(max_results=2)
tools = load_tools(
    ["llm-math"], 
    llm=model #第一步初始化的模型
)
tools.append(search)

# 初始化代理
agent= initialize_agent(
    tools, #第二步加载的工具
    model, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)
agent("计算300的25%")



> Entering new AgentExecutor chain...
Could not parse LLM output: To find 25% of 300 using the Calculator tool, we can represent this as a json blob:

Question: To calculate what percentage of 300 is represented by 25%
Thought: We need to perform the calculation with the given percentage and input number.
Action:
{
  "action": "Calculator",
  "action_input": {
    "expression": "0.25 * 300"
  }
}

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: To calculate 25% of 300 using the Calculator tool, we perform the following action:

Action:
{
  "action": "Calculator",
  "action_input": {
    "expression": "0.25 * 300"
  }
}


For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: To calculate 25% of 300 using the Calcu

{'input': '计算300的25%', 'output': '75'}